In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


In [7]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 48.8534,
	"longitude": 2.3488,
	"start_date": "2023-01-01",
	"end_date": "2023-12-31",
	"hourly": "temperature_2m",
	"models": "meteofrance_seamless"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe_past_forecast_Paris = pd.DataFrame(data = hourly_data)
hourly_dataframe_past_forecast_Paris.dropna(inplace=True)

Coordinates 48.849998474121094°N 2.34999942779541°E
Elevation 43.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [9]:
print(hourly_dataframe_past_forecast_Paris)
hourly_dataframe_past_forecast_Paris.to_csv('hourly_dataframe_past_forecast_Paris.csv')

                          date  temperature_2m
0    2023-01-01 00:00:00+00:00       15.938999
1    2023-01-01 01:00:00+00:00       15.188999
2    2023-01-01 02:00:00+00:00       14.839000
3    2023-01-01 03:00:00+00:00       14.438999
4    2023-01-01 04:00:00+00:00       13.988999
...                        ...             ...
8755 2023-12-31 19:00:00+00:00        8.011000
8756 2023-12-31 20:00:00+00:00        7.511000
8757 2023-12-31 21:00:00+00:00        7.411000
8758 2023-12-31 22:00:00+00:00        7.661000
8759 2023-12-31 23:00:00+00:00        7.561000

[8755 rows x 2 columns]
